# TP 3 : Programmation dynamique

### Questions

    1. Donner une courte argumentation de l'adéquation du résultat avec l'instance résolue.
    
Avec l'algorithme de programmation dynamique, on doit trouver que le coût maximal est de 65, en mettant les objets 2 et 3 (de 4kg et 5kg). C'est bien ce que l'on retrouve ici.

    2. Expliquer le fonctionnement de votre algorithme.

Cet algorithme a pour objectif de résoudre le problème du sac à dos, c'est-à-dire de trouver la combinaison d'objets à mettre dans un sac ayant une capacité donnée afin d'obtenir le profit maximum.

Pour cela, on crée d'abord deux tableaux : un pour stocker les profits et l'autre pour stocker la liste des objets qui ont été sélectionnés. Ensuite, on classe les objets par ordre croissant de poids. Et on crée une liste où on mettra la solution mémorisée (les objets qu'on met dans le sac).

On initialise la première ligne (ie celle pour l'objet de plus faible poids) du tableau des profits en fonction du poids de l'objet : si le poids de l'objet est supérieur à la capacité du sac, alors le profit est nul, sinon, le profit est égal au prix de l'objet et l'objet est ajouté à la liste des objets sélectionnés.

Puis, on remplit le reste du tableau en parcourant chaque ligne (correspondant à un objet) et chaque colonne (correspondant à une capacité du sac). Pour chaque case, on compare le profit obtenu en prenant l'objet et celui obtenu en ne le prenant pas. Si le profit est meilleur en prenant l'objet, celui-ci est ajouté à la liste des objets sélectionnés et le profit est mis à jour dans le tableau. Sinon, le profit reste inchangé et la liste des objets sélectionnés est copiée de la ligne précédente.

Enfin, l'algorithme affiche le tableau des profits et retourne le profit maximum et la liste des objets sélectionnés.
    
    3. Tester l'algorithme sur plusieurs instances de tailles différentes fournies dans le TP2, donner les solutions obtenues et les valeurs de fonction-objectif. Comparer avec celles obtenues avec le branch-and-bound implémenté lors du TP2.
    
Pour l'exemple du TP2 avec les sac à dos, on trouve les mêmes résultats : $BestProfit = 65$ avec les objets de 4kg et 5kg.
    
Par exemple, pour les fichiers suivant on obtient les bons profits, on en conclut que l'algorithme fonctionne bien.

- profit_ceiling/knapPI_15_20_1000_1_-999.opb.txt

Optimal value = 999 -> C'est ce qui était attendu

Optimal x = Any["7", "237", "334", "416"]

- weakly_correlated/knapPI_2_50_1000_1_-1396.opb.txt

Optimal value = 1396

Optimal x = Any["70", "72", "97", "122", "213", "421"]

- subset_sum/knapPI_6_100_1000_5_-2399.opb.txt

Optimal value = 2399

Optimal x = Any["1", "6", "6", "8", "9", "9", "15", "18", "20", "20", "23", "27", "28", "29", "32", "33", "34", "35", "39", "39", "43", "44", "45", "46", "47", "53", "57", "58", "59", "59", "61", "63", "64", "65", "70", "70", "70", "70", "72", "81", "94", "94", "94", "96", "97", "98", "98", "100"]

- uncorrelated_span/knapPI_11_50_1000_1_-1428.opb.txt

Optimal value = 1428

Optimal x = Any["66", "66", "132", "198", "198", "264"]
    

### Récupération des données

In [2]:
function readKnaptxtInstance(filename)
    price=[]
    weight=[]
    KnapCap=[]
    open(filename) do f
        for i in 1:3
            tok = split(readline(f))
            if(tok[1] == "ListPrices=")
                for i in 2:(length(tok)-1)
                    push!(price,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "ListWeights=")
                for i in 2:(length(tok)-1)
                    push!(weight,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "Capacity=")
                push!(KnapCap, parse(Int64, tok[2]))
            else
                println("Unknown read :", tok)
            end 
        end
    end
    capacity=KnapCap[1]
    return price, weight, capacity
end

readKnaptxtInstance (generic function with 1 method)

### Création du tableau

In [3]:
function CreateTab(price, weight, capacity)
    
    ### Création des tableaux
    n = length(price)
    tabPrix = zeros(Int64, n, capacity) 
    tabPoids = [[] for i=1:n, j=1:capacity]
    
    ### Classer les objets par ordre croissant de poids
    order = sortperm(weight)
    weight = weight[order]
    price = price[order]
    
    ### Liste de la solution mémorisée (les objets qu'on met dans le sac)
    obj = []
    
    ### Initialiser la première ligne
    for j in 1:capacity
        if j < weight[1]
            tabPrix[1,j] = 0
        else
            tabPrix[1,j] = price[1]
            tabPoids[1,j] = [string(weight[1])]
        end
    end
    
    ### Remplir le tableau à partir de la 2e ligne
    for i in 2:n
        for j in 1:capacity
            if (j-weight[i]) < 0
                tabPrix[i,j] = tabPrix[i-1, j]
                tabPoids[i,j] = tabPoids[i-1, j]
            elseif (j-weight[i]) == 0
                tabPrix[i,j] = max(tabPrix[i-1,j], price[i])
                if tabPrix[i,j] == price[i]
                    tabPoids[i,j] = [string(weight[i])]
                else
                    tabPoids[i,j] = tabPoids[i-1,j]
                end
            else 
                tabPrix[i,j] = max(tabPrix[i-1, j], tabPrix[i-1, j-weight[i]] + price[i])
                if tabPrix[i,j] == tabPrix[i-1, j]
                    tabPoids[i,j] = tabPoids[i-1,j]
                else
                    tabPoids[i,j] = append!(copy(tabPoids[i-1,j-weight[i]]),[string(weight[i])])
                end
            end 
        end
    end
    
    display(tabPrix)
    #show(stdout, "text/plain", tabPoids)

    BestProfit = tabPrix[n, capacity]
    BestSolution = tabPoids[n, capacity]

    return BestProfit, BestSolution
end

CreateTab (generic function with 1 method)

In [4]:
price = [42, 25, 40, 12]
weight = [7,5,4,3]
capacity = 10

CreateTab(price, weight, capacity)

4×10 Matrix{Int64}:
 0  0  12  12  12  12  12  12  12  12
 0  0  12  40  40  40  52  52  52  52
 0  0  12  40  40  40  52  52  65  65
 0  0  12  40  40  40  52  52  65  65

(65, Any["4", "5"])

### Résolution

In [5]:
function SolveKnapInstance(filename)

    price, weight, capacity = readKnaptxtInstance(filename)
    
    BestProfit, BestSolution = CreateTab(price, weight, capacity) 
    
    return BestProfit, BestSolution

end

SolveKnapInstance (generic function with 1 method)

### Affichage du résultat final

In [8]:
BestProfit, BestSolution = SolveKnapInstance("instancesETU/KNAPnewformat/test.opb.txt")
println("\n******\n\nOptimal value = ", BestProfit, "\n\nOptimal x = ", BestSolution)

4×10 Matrix{Int64}:
 0  0  12  12  12  12  12  12  12  12
 0  0  12  40  40  40  52  52  52  52
 0  0  12  40  40  40  52  52  65  65
 0  0  12  40  40  40  52  52  65  65


******

Optimal value = 65

Optimal x = Any["4", "5"]
